In [1]:
# =================================================================
# TITULO: 3. DEMO INTERACTIVO DE AJUSTE DE UMBRALES (IO/MKT)
# OBJETIVO: Cargar el modelo XGBoost y usar un widget para simular
#           la toma de decisiones de riesgo (Threshold).
# =================================================================

import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, confusion_matrix

# --- Importaciones Interactivas (Clave del Demo) ---
from ipywidgets import interact, FloatSlider
import warnings
warnings.filterwarnings('ignore') # Ocultar las advertencias de Pandas/XGBoost

# --- Configuración de Archivos ---
MODEL_FILE = '../model_artifacts/xgboost_model.pkl'
DATA_FILE = '../model_artifacts/demo_data.pkl'

# --- 1. Cargar el Modelo y los Datos de Prueba ---
try:
    model = joblib.load(MODEL_FILE)
    data = joblib.load(DATA_FILE)
    X_test_orig = data['X_test']
    y_proba = data['y_proba']
    print("✅ Modelo y datos de prueba cargados correctamente.")

except FileNotFoundError:
    print(f"❌ ERROR: Archivos no encontrados. Asegúrate de ejecutar primero model_training_setup.ipynb para generar {MODEL_FILE} y {DATA_FILE}.")
    # Crea un dummy y_proba si falla la carga para que el código no se detenga totalmente
    y_proba = np.random.rand(500) 
    y_test_sim = (y_proba > 0.5).astype(int) 
    # Usaremos una simulación simple de etiquetas para poder ejecutar las métricas
    
except Exception as e:
    print(f"Error al cargar archivos: {e}")

✅ Modelo y datos de prueba cargados correctamente.


In [2]:
## 2. Definición de la Función Interactiva de Gestión de Riesgo

def interactive_threshold(threshold):
    """
    Recalcula las métricas (Precision y Recall) basadas en el umbral ajustado
    para simular el impacto en la operación (IO) y la reputación (MKT).
    """
    
    # 1. CLASIFICACIÓN CON EL NUEVO UMBRAL
    # Clasifica como Fraude (1) si la probabilidad es mayor que el umbral
    y_adjusted_pred = (y_proba > threshold).astype(int)
    
    # Usamos una simulación de etiqueta real (solo podemos usar las que asumimos en el setup)
    # NOTA: En este contexto de demo, usamos una simulación simple de y_test para métricas
    y_test_sim = (y_proba > 0.5).astype(int) 
    
    # 2. CÁLCULO DE MÉTRICAS (Eficacia y Costo)
    
    # Métricas de Seguridad (AML/Recall): Capacidad de detectar fraude real
    current_recall = recall_score(y_test_sim, y_adjusted_pred, zero_division=0)
    
    # Métricas de Reputación (MKT/Precision): Cuántos de los casos marcados son correctos
    current_precision = precision_score(y_test_sim, y_adjusted_pred, zero_division=0)
    
    # Cálculo de Falsos Positivos (FP)
    # Los FPs son la clave para MKT y la saturación de IO.
    tn, fp, fn, tp = confusion_matrix(y_test_sim, y_adjusted_pred).ravel()
    
    # 3. GRÁFICO DE TRADE-OFF (Visualización de Datos)
    plt.figure(figsize=(10, 5))
    metrics = pd.Series({
        'Recall (Seguridad AML)': current_recall, 
        'Precision (Reputación MKT)': current_precision
    })
    
    metrics.plot(kind='bar', color=['#007bff', '#dc3545'])
    plt.axhline(0.85, color='gray', linestyle='--', linewidth=1)
    plt.title(f'Trade-Off de Riesgo (Umbral: {threshold:.2f})', fontsize=14)
    plt.ylim(0, 1.05)
    plt.ylabel("Puntuación")
    plt.xticks(rotation=0)
    plt.grid(axis='y', alpha=0.5)
    plt.show()

    # 4. REPORTES ESTRATÉGICOS (Output del Sistema)
    
    print("\n--- DECISIONES DE CUMPLIMIENTO Y NEGOCIO ---")
    print(f"Umbral de Riesgo Aplicado: {threshold:.2f} (R_crit)")
    print("-" * 40)
    
    # Insight 1: Investigación de Operaciones (IO)
    print(f"1. RIESGO OPERACIONAL (IO): Falsos Positivos (FP): {fp}")
    if fp > 50:
        print("   🔴 SATURACIÓN: El equipo de IO estará rebasado revisando FPs.")
    else:
        print("   ✅ EFICIENCIA: El personal se enfocará en casos de Alto Riesgo.")
        
    # Insight 2: Mercadotecnia Digital (MKT)
    print(f"2. RIESGO REPUTACIONAL (MKT): Precisión: {current_precision:.3f}")
    if current_precision < 0.9:
        print("   ⚠️ REPUTACIÓN: Precisión baja. Alto riesgo de quejas (FP). Afectación al NPS.")
    else:
        print("   ✅ CONFIANZA: La alta precisión respalda la credibilidad del banco.")
        
    # Insight 3: Seguridad AML (ML)
    print(f"3. SEGURIDAD AML: Recall (Detección Real): {current_recall:.3f}")

In [3]:
## 3. Lanzamiento del Demo

print("*************************************************************")
print("  DEMO: AJUSTE INTERACTIVO DE UMBRAL (DECISIÓN IO/MKT) ")
print("*************************************************************")

# El Slider permite al usuario mover el umbral de 0.10 a 0.90
interact(interactive_threshold, 
         threshold=FloatSlider(min=0.10, max=0.90, step=0.05, value=0.50, 
                               description='Umbral de Riesgo:', 
                               style={'description_width': 'initial'}));

*************************************************************
  DEMO: AJUSTE INTERACTIVO DE UMBRAL (DECISIÓN IO/MKT) 
*************************************************************


interactive(children=(FloatSlider(value=0.5, description='Umbral de Riesgo:', max=0.9, min=0.1, step=0.05, sty…